<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
from __future__ import print_function  

import scipy.sparse as sp  
import numpy as np  
from scipy.sparse.linalg.eigen.arpack import eigsh, ArpackNoConvergence  # 稀疏矩阵中查找特征值/特征向量的函数

from keras import activations, initializers, constraints
from keras import regularizers
from keras.engine import Layer
import keras.backend as K
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

import time

In [3]:
def encode_onehot(labels):
    """
    将标签转换为one-hot编码形式
    
    """
    classes = set(labels)
    classes_dict = {
        c: np.identity(len(classes))[i, :]
        for i, c in enumerate(classes)
    }  # np.identity() 创建一个单位对角阵

    labels_onehot = np.array(
        list(map(classes_dict.get, labels)),
        dtype=np.int32)  # map(function, iterable)： 对每个 label，应用 class_dict()
    return labels_onehot


def load_data(path="data/cora/", dataset="cora"):
    """
    Load citation network dataset 
    
    (cora only for now)
    """

    print('Loading {} dataset...'.format(dataset))

    ### 读取样本id，特征和标签
    idx_features_labels = np.genfromtxt(
        "{}{}.content".format(path, dataset),
        dtype=np.dtype(str))  # np.genfromtxt()生成 array
    features = sp.csr_matrix(idx_features_labels[:, 1:-1],
                             dtype=np.float32)  # 提取样本的特征，并将其转换为csr矩阵
    labels = encode_onehot(
        idx_features_labels[:, -1])  # 提取样本的标签，并将其转换为one-hot编码形式
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)  # 样本的id数组
    idx_map = {j: i for i, j in enumerate(idx)}  # 创建一个字典储存数据id

    ### 读取样本之间关系 ： 连边
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(
                         edges_unordered.shape)  # 无序边  map 成为有序
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)  # 构建图的邻接矩阵
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(
        adj.T > adj)  # 将非对称邻接矩阵转变为对称邻接矩阵

    print('Dataset has {} nodes, {} edges, {} features.'.format(
        adj.shape[0], edges.shape[0], features.shape[1]))

    return features.todense(), adj, labels  # 返回特征的密集矩阵表示、邻接矩阵和标签的one-hot编码


def normalize_adj(adj, symmetric=True):
    """
    对邻接矩阵进行归一化处理
    对称归一化 ： D^(-1/2) * A * D^(-1/2)
    非对称归一化： D^(-1) * A
    """

    if symmetric:
        d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(),
                     0)  # D^(-1/2)
        # tocsr()函数将矩阵转化为压缩稀疏行矩阵
        a_norm = adj.dot(d).transpose().dot(d).tocsr(
        )  #  d 和 A 是对称矩阵 ： (A*  d)^T * d = d^T * A^T  * d= d * A * d

    else:
        d = sp.diags(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = d.dot(adj).tocsr()  # D^(-1) * A
    return a_norm


def preprocess_adj(adj, symmetric=True):
    """
    在邻接矩阵中加入自连接
    """
    adj = adj + sp.eye(adj.shape[0])  # \tilde{A}=A+I_{N}
    adj = normalize_adj(adj, symmetric)  # renormalization trick
    return adj


def sample_mask(idx, l):
    """
    :param idx: 有标签样本的索引列表
    :param l: 所有样本数量
    :return: 布尔类型数组，其中有标签样本所对应的位置为True，无标签样本所对应的位置为False
    
    example : 
    
    >>> mask = np.zeros(5)
    >>> list_ = [1,3]
    >>> mask[list_]  = 1 
    >>> np.array(mask,dtype = np.bool)
    
    return : array([False,  True, False,  True, False])
    """
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def get_splits(y):
    """
    数据集划分：
    
    y : 标签数据 即 load_data() 函数 返回的 labels 数据 
    数据格式  (2708 , 7 ) 2708 个数据，一共 7 个 class ， one-hot 维度 7 维 
    """

    idx_train = range(140)  # ??? 少一部分数据啊
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    y_train = np.zeros(y.shape, dtype=np.int32) # 初始化 (2708,7) size 的全 0 矩阵。
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train] #  取 y 矩阵前 140 行， 其余行赋值为 0 
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]

    # 训练数据的样本掩码
    train_mask = sample_mask(idx_train, y.shape[0]) # 前 140行为 true， 后面为 false 的向量
    return y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask


def categorical_crossentropy(preds, labels):
    """
    损失函数计算： 所有有标签的数据，主要用于训练时进行梯度下降 ： 
    
    L=Y*ln(Z)
    
    param preds: 模型对样本的输出数组
    param labels: 样本的one-hot标签数组
    return: 样本的平均交叉熵损失
    """
    return np.mean(-np.log(np.extract(
        labels, preds)))  # np.extract(condition, x)函数，根据某个条件从数组中抽取元素


def accuracy(preds, labels):
    """
    计算准确率
    """
    # np.argmax(x,1) 每一行元素最大值索引  : 相当于将 one-hot 转成一个数值 [0,1,0,0,0] = 1
    # np.equal(x1, x2) 比较 array 对应元素是否相等 返回 np.array([ True,  True, False])
    # np.mean(np.array([ True,  True, False])) = 0.666
    return np.mean(np.equal(np.argmax(labels, 1), np.argmax(preds, 1)))


def evaluate_preds(preds, labels, indices):
    """
     评估样本划分的损失函数和准确率
     
    :param preds:对于样本的预测值
    :param labels:样本的标签one-hot向量
    :param indices:样本的索引集合
    :return:交叉熵损失函数列表、准确率列表
    """
    split_loss = list()
    split_acc = list()

    for y_split, idx_split in zip(labels, indices):
        # 计算每一个样本划分的交叉熵损失函数
        split_loss.append(
            categorical_crossentropy(preds[idx_split], y_split[idx_split]))
        # 计算每一个样本划分的准确率
        split_acc.append(accuracy(preds[idx_split], y_split[idx_split]))

    return split_loss, split_acc


def normalized_laplacian(adj, symmetric=True):
    """
    对拉普拉斯矩阵进行归一化处理
    """
    # 对称归一化的邻接矩阵，D ^ (-1/2) * A * D ^ (-1/2)
    adj_normalized = normalize_adj(adj, symmetric)
    # 得到对称规范化的图拉普拉斯矩阵，L = I - D ^ (-1/2) * A * D ^ (-1/2)
    laplacian = sp.eye(adj.shape[0]) - adj_normalized
    return laplacian


def rescale_laplacian(laplacian):
    """
    特征值调整到 [-1,1]
    """
    try:
        print(
            'Calculating largest eigenvalue of normalized graph Laplacian...')
        # 计算对称归一化图拉普拉斯矩阵的最大特征值
        largest_eigval = eigsh(laplacian,
                               1,
                               which='LM',
                               return_eigenvectors=False)[0]
    # 如果计算过程不收敛 : 直接设置为 2
    except ArpackNoConvergence:
        print(
            'Eigenvalue calculation did not converge! Using largest_eigval=2 instead.'
        )
        largest_eigval = 2

    # 调整后的对称归一化图拉普拉斯矩阵，L~ = 2 / Lambda * L - I
    scaled_laplacian = (2. / largest_eigval) * laplacian - sp.eye(
        laplacian.shape[0])
    return scaled_laplacian


def chebyshev_polynomial(X, k):
    """
    Calculate Chebyshev polynomials up to order k. 
    Return a list of sparse matrices.
    """
    print("Calculating Chebyshev polynomials up to order {}...".format(k))

    T_k = list()
    T_k.append(sp.eye(X.shape[0]).tocsr())  # T0(X) = I
    T_k.append(X)  # T1(X) = L~

    def chebyshev_recurrence(T_k_minus_one, T_k_minus_two, X):
        """
        定义切比雪夫递归公式
        :param T_k_minus_one: T(k-1)(L~)
        :param T_k_minus_two: T(k-2)(L~)
        :param X: L~
        :return: Tk(L~)
        """
        X_ = sp.csr_matrix(X, copy=True)  # 将输入转化为csr矩阵（压缩稀疏行矩阵）
        # 递归公式：Tk(L~) = 2L~ * T(k-1)(L~) - T(k-2)(L~)
        return 2 * X_.dot(T_k_minus_one) - T_k_minus_two

    for i in range(2, k + 1):
        T_k.append(chebyshev_recurrence(T_k[-1], T_k[-2], X))

    # 返回切比雪夫多项式列表
    return T_k


def sparse_to_tuple(sparse_mx):
    """
     将稀疏矩阵转化为元组表示
    """
    if not sp.isspmatrix_coo(sparse_mx):
        # 将稀疏矩阵转化为coo矩阵形式
        # coo矩阵采用三个数组分别存储行、列和非零元素值的信息
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack(
        (sparse_mx.row, sparse_mx.col)).transpose()  # 获取非零元素的位置索引
    values = sparse_mx.data  # 获取矩阵的非零元素
    shape = sparse_mx.shape  # 获取矩阵的形状

    return coords, values, shape


 **GCN 层**
 
 

<img style="float:;" src="https://tva1.sinaimg.cn/large/007S8ZIlly1ge846nffkhj30wt0gyjvd.jpg" width="40%">




In [4]:
class GraphConvolution(Layer):
    """
    Basic graph convolution layer as in 
    """
    def __init__(self,
                 units,
                 support=1,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            #不同 keras 版本
            kwargs['input_shape'] = (kwargs.pop('input_dim'), )
        super(GraphConvolution, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.supports_masking = True
        self.support = support  # 多个图的情况
        assert support >= 1

    ### 参考 https://keras.io/zh/layers/writing-your-own-keras-layers/
    def compute_output_shape(self, input_shapes):
        """
        层更改了输入张量的形状
        让Keras能够自动推断各层的形状
        """
        features_shape = input_shapes[0] #  n * c ， 输入列表的第一部分
        output_shape = (features_shape[0], self.units) # 输出  n * f (输出特征维度)
        return output_shape  # (batch_size, output_dim)

    def build(self, input_shapes):
        """
        定义层中的权重
        为该层创建一个可训练的权重
        """
        features_shape = input_shapes[0]  #  n * c ， 输入列表的第一部分
        assert len(features_shape) == 2
        input_dim = features_shape[1] #  权重 维度 ： c * f（unit）

        self.kernel = self.add_weight(shape=(input_dim * self.support,
                                             self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        # 如果存在偏置
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units, ),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True  # 必须设定self.bulit = True

    def call(self, inputs, mask=None):
        """
        编写层的功能逻辑
        """
        features = inputs[0]  # 特征
        basis = inputs[1:]  # 对称归一化的邻接矩阵， 输入列表的第二部分

        supports = list()  # 多个图的情况
        for i in range(self.support):
            supports.append(K.dot(basis[i], features))  # A * X
        supports = K.concatenate(supports, axis=1)  # 将多个图的结果按行拼接
        output = K.dot(supports, self.kernel)  # A * X * W

        if self.bias:
            # A * X * W + b
            output += self.bias
        return self.activation(output)

    def get_config(self):
        """
        当前层的配置信息
        dictionary containing the configuration of the model
        """
        config = {
            'units':
            self.units,
            'support':
            self.support,
            'activation':
            activations.serialize(self.activation),
            'use_bias':
            self.use_bias,
            'kernel_initializer':
            initializers.serialize(self.kernel_initializer),
            'bias_initializer':
            initializers.serialize(self.bias_initializer),
            'kernel_regularizer':
            regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer':
            regularizers.serialize(self.bias_regularizer),
            'activity_regularizer':
            regularizers.serialize(self.activity_regularizer),
            'kernel_constraint':
            constraints.serialize(self.kernel_constraint),
            'bias_constraint':
            constraints.serialize(self.bias_constraint)
        }

        base_config = super(GraphConvolution, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [5]:
DATASET = 'cora'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 20000
PATIENCE = 10  # early stopping patience

# Get data
X, A, y = load_data(
    dataset=DATASET)  # X: 特征（2708，2708）、 A: 邻接矩阵 （2708，2708）、 y: 标签 （2708，7）
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(
    y)
X /= X.sum(1).reshape(-1, 1)  # Normalize  X : (2708, 1433)

if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
    print('Using local pooling filters...')
    A_ = preprocess_adj(A, SYM_NORM)  # 加入 self-loop 的邻接矩阵
    support = 1
    graph = [X, A_] # 按照自定义的 GCN 层， 输入应该是包含两部分的 列表
    G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    print('Using Chebyshev polynomial basis filters...')
    L = normalized_laplacian(A, SYM_NORM)  # 对拉普拉斯矩阵进行归一化处理
    L_scaled = rescale_laplacian(L)  # L~ = 2 / Lambda * L - I
    T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)  # 2介切比雪夫多项式
    support = MAX_DEGREE + 1
    graph = [X] + T_k  # 列表相加
    G = [
        Input(shape=(None, None), batch_shape=(None, None), sparse=True)
        for _ in range(support)
    ]

else:
    raise Exception('Invalid filter type.')

### 函数式建模， 把 层当作 函数 使用
X_in = Input(shape=(X.shape[1], ))  # 输入维度，不包括 batch_size， 即 输入的特征维度

# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16,
                     support,
                     activation='relu',
                     kernel_regularizer=l2(5e-4))([H] + G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H] + G)

# Compile model
model = Model(inputs=[X_in] + G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH + 1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(
        graph,
        y_train,
        sample_weight=
        train_mask,  # 向sample_weight 用于对损失函数进行加权 , 因此只有训练数据对损失函数有贡献
        batch_size=A.shape[0],  # 每次的 batch_size  ,
        epochs=1,
        shuffle=False,
        verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.
Using local pooling filters...
Epoch: 0001 train_loss= 1.9363 train_acc= 0.2214 val_loss= 1.9395 val_acc= 0.1600 time= 1.0304
Epoch: 0002 train_loss= 1.9225 train_acc= 0.2000 val_loss= 1.9290 val_acc= 0.1600 time= 0.0714
Epoch: 0003 train_loss= 1.9081 train_acc= 0.2000 val_loss= 1.9182 val_acc= 0.1567 time= 0.0608
Epoch: 0004 train_loss= 1.8933 train_acc= 0.2000 val_loss= 1.9069 val_acc= 0.1567 time= 0.0636
Epoch: 0005 train_loss= 1.8786 train_acc= 0.2000 val_loss= 1.8960 val_acc= 0.1567 time= 0.0659
Epoch: 0006 train_loss= 1.8631 train_acc= 0.2000 val_loss= 1.8847 val_acc= 0.1567 time= 0.0626
Epoch: 0007 train_loss= 1.8474 train_acc= 0.2000 val_loss= 1.8735 val_acc= 0.1567 time= 0.0598
Epoch: 0008 train_loss= 1.8327 train_acc= 0.2000 val_loss= 1.8631 val_acc= 0.1567 time= 0.0596
Epoch: 0009 train_loss= 1.8182 train_acc= 0.2000 val_loss= 1.8532 val_acc= 0.1567 time= 0.0664
Epoch: 0010 train_loss= 1.8039 train_ac

Epoch: 0089 train_loss= 0.9864 train_acc= 0.8357 val_loss= 1.2309 val_acc= 0.7300 time= 0.0718
Epoch: 0090 train_loss= 0.9781 train_acc= 0.8357 val_loss= 1.2247 val_acc= 0.7300 time= 0.0709
Epoch: 0091 train_loss= 0.9699 train_acc= 0.8429 val_loss= 1.2183 val_acc= 0.7300 time= 0.0796
Epoch: 0092 train_loss= 0.9618 train_acc= 0.8429 val_loss= 1.2121 val_acc= 0.7300 time= 0.0708
Epoch: 0093 train_loss= 0.9538 train_acc= 0.8357 val_loss= 1.2056 val_acc= 0.7300 time= 0.0704
Epoch: 0094 train_loss= 0.9460 train_acc= 0.8357 val_loss= 1.1992 val_acc= 0.7267 time= 0.0715
Epoch: 0095 train_loss= 0.9384 train_acc= 0.8429 val_loss= 1.1929 val_acc= 0.7267 time= 0.0818
Epoch: 0096 train_loss= 0.9310 train_acc= 0.8429 val_loss= 1.1869 val_acc= 0.7300 time= 0.0785
Epoch: 0097 train_loss= 0.9237 train_acc= 0.8429 val_loss= 1.1809 val_acc= 0.7333 time= 0.0739
Epoch: 0098 train_loss= 0.9163 train_acc= 0.8429 val_loss= 1.1753 val_acc= 0.7433 time= 0.0730
Epoch: 0099 train_loss= 0.9090 train_acc= 0.8429 v

Epoch: 0178 train_loss= 0.5509 train_acc= 0.9286 val_loss= 0.8899 val_acc= 0.7967 time= 0.0644
Epoch: 0179 train_loss= 0.5483 train_acc= 0.9286 val_loss= 0.8889 val_acc= 0.8033 time= 0.0618
Epoch: 0180 train_loss= 0.5459 train_acc= 0.9286 val_loss= 0.8883 val_acc= 0.8033 time= 0.0621
Epoch: 0181 train_loss= 0.5438 train_acc= 0.9286 val_loss= 0.8871 val_acc= 0.8033 time= 0.0612
Epoch: 0182 train_loss= 0.5414 train_acc= 0.9357 val_loss= 0.8843 val_acc= 0.8033 time= 0.0664
Epoch: 0183 train_loss= 0.5390 train_acc= 0.9357 val_loss= 0.8818 val_acc= 0.8033 time= 0.0722
Epoch: 0184 train_loss= 0.5366 train_acc= 0.9357 val_loss= 0.8789 val_acc= 0.8067 time= 0.0679
Epoch: 0185 train_loss= 0.5345 train_acc= 0.9357 val_loss= 0.8766 val_acc= 0.8033 time= 0.0666
Epoch: 0186 train_loss= 0.5325 train_acc= 0.9357 val_loss= 0.8742 val_acc= 0.8033 time= 0.0737
Epoch: 0187 train_loss= 0.5300 train_acc= 0.9286 val_loss= 0.8730 val_acc= 0.8000 time= 0.0772
Epoch: 0188 train_loss= 0.5273 train_acc= 0.9286 v

Epoch: 0267 train_loss= 0.3965 train_acc= 0.9786 val_loss= 0.7746 val_acc= 0.8033 time= 0.0646
Epoch: 0268 train_loss= 0.3952 train_acc= 0.9714 val_loss= 0.7742 val_acc= 0.8033 time= 0.0634
Epoch: 0269 train_loss= 0.3938 train_acc= 0.9714 val_loss= 0.7737 val_acc= 0.8033 time= 0.0640
Epoch: 0270 train_loss= 0.3920 train_acc= 0.9643 val_loss= 0.7743 val_acc= 0.8033 time= 0.0627
Epoch: 0271 train_loss= 0.3908 train_acc= 0.9643 val_loss= 0.7758 val_acc= 0.8067 time= 0.0670
Epoch: 0272 train_loss= 0.3898 train_acc= 0.9643 val_loss= 0.7779 val_acc= 0.8000 time= 0.0639
Epoch: 0273 train_loss= 0.3885 train_acc= 0.9571 val_loss= 0.7783 val_acc= 0.8000 time= 0.0622
Epoch: 0274 train_loss= 0.3866 train_acc= 0.9643 val_loss= 0.7774 val_acc= 0.8000 time= 0.0635
Epoch: 0275 train_loss= 0.3844 train_acc= 0.9714 val_loss= 0.7749 val_acc= 0.8033 time= 0.0643
Epoch: 0276 train_loss= 0.3825 train_acc= 0.9714 val_loss= 0.7725 val_acc= 0.8067 time= 0.0623
Epoch: 0277 train_loss= 0.3813 train_acc= 0.9786 v

Epoch: 0355 train_loss= 0.3112 train_acc= 0.9714 val_loss= 0.7121 val_acc= 0.8133 time= 0.0669
Epoch: 0356 train_loss= 0.3090 train_acc= 0.9714 val_loss= 0.7118 val_acc= 0.8133 time= 0.0616
Epoch: 0357 train_loss= 0.3079 train_acc= 0.9786 val_loss= 0.7134 val_acc= 0.8267 time= 0.0625
Epoch: 0358 train_loss= 0.3085 train_acc= 0.9786 val_loss= 0.7184 val_acc= 0.8267 time= 0.0655
Epoch: 0359 train_loss= 0.3112 train_acc= 0.9857 val_loss= 0.7263 val_acc= 0.8233 time= 0.0629
Epoch: 0360 train_loss= 0.3139 train_acc= 0.9857 val_loss= 0.7324 val_acc= 0.8233 time= 0.0647
Epoch: 0361 train_loss= 0.3139 train_acc= 0.9857 val_loss= 0.7318 val_acc= 0.8200 time= 0.0637
Epoch: 0362 train_loss= 0.3117 train_acc= 0.9857 val_loss= 0.7264 val_acc= 0.8200 time= 0.0638
Epoch: 0363 train_loss= 0.3092 train_acc= 0.9857 val_loss= 0.7202 val_acc= 0.8200 time= 0.0627
Epoch: 0364 train_loss= 0.3076 train_acc= 0.9857 val_loss= 0.7137 val_acc= 0.8200 time= 0.0615
Epoch: 0365 train_loss= 0.3086 train_acc= 0.9786 v

In [6]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:", "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

Test set results: loss= 0.7351 accuracy= 0.8220


In [34]:
model.summary() # 模型结构
model.layers[1].name 
weights, biases = model.layers[3].get_weights() # 模型权重

array([[ 1.2420109e-01,  1.7771399e-11, -9.4312154e-02, ...,
         8.3028547e-02,  1.3846582e-01, -1.6981850e-11],
       [ 6.8800889e-02, -4.1161324e-11, -6.7035429e-02, ...,
         2.0541064e-02, -1.1935651e-01,  7.6480794e-11],
       [ 8.9236625e-02, -1.4951953e-11,  1.9362820e-02, ...,
         5.1603135e-02,  4.7853532e-01, -7.1407248e-11],
       ...,
       [-6.3166186e-02,  5.2331157e-11,  1.9311365e-01, ...,
        -3.1927913e-02,  1.4044446e-01, -4.1707922e-12],
       [-1.1241451e-01,  2.2368515e-11,  1.7177092e-01, ...,
        -3.6130950e-02,  3.7736788e-01,  5.7867818e-11],
       [ 1.8448421e-03, -1.4665644e-10,  5.2221458e-02, ...,
        -3.6790878e-02, -3.4460209e-02, -2.3777178e-12]], dtype=float32)

In [ ]:
model.save("my_keras_model.h5") # 保存模型
model = keras.models.load_model("my_keras_model.h5")

save_weights() # 保存权重
load_weights()